In [ ]:
import sys, os
cwd=os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, cwd)
import utils.s3_utils as s3
import pandas as pd
import shapefile as shp
from shapely.geometry import Point
from shapely.geometry.polygon import Point, Polygon
import geopandas as gpd
import re
import matplotlib.pyplot as plt
import rioxarray as rxr
import xarray as xr
from scipy.spatial.distance import cdist
import folium
import utils.processing as pr
import os
import rasterio
pd.set_option('display.max_columns', None)


In [ ]:


label_dict = {
    'tg' : 'meanTemp',
    'fg' : 'windSpeed',
    'tn' : 'minTemp',
    'tx' : 'maxTemp',
    'rr' : 'Precip',
    'hu' : 'relHumidity', #v29 humidity from e-obs not present for 2011-2023, so pulled v28.0
    'qq' : 'meanRadiation'
}

wdf_agg_list=[]
for data_col in label_dict.keys():
    print(data_col)
    wdf_list = []
    for timeframe in ['1995-2010', "2011-2023"]:
        weather_filename = f"../../data/climate/0.1deg/eobs-eu-climate-{timeframe}/{data_col}_ens_mean_0.1deg_reg_{timeframe}_v29.0e.nc"
        print(weather_filename)
        ds = xr.open_dataset(weather_filename, engine='h5netcdf')
        wdf = ds.to_dataframe()
        del ds
        wdf = wdf.reset_index()
        wdf = wdf.dropna()
        wdf = wdf[wdf.time >= '1999-12-01']
        wdf_list.append(wdf)
    wdf = pd.concat(wdf_list).reset_index(drop=True)
    display(wdf.info())
    del wdf_list
    for year in range(2000,2023):
        print(year)
        wdf_yr = wdf[(f'{year-1}-12-01'<= wdf.time) & (wdf.time < f'{year}-12-01')]
        path_to_write_file=f'../../data/climate/0.1deg/eobs-eu-climate-parquet-2000-2023/' + f'{data_col}/{data_col}_ens_mean_0.1deg_reg_{year}_v29.0e.parquet'
        wdf_yr.to_parquet(path_to_write_file, compression ='snappy')